<font size=50 color=darkblue>Graph Coloring using Big-M Transformation</font>

# Problem modelling with DOcplex

## Import necessary modules

- DOcplex will be used to solve the Graph-Coloring MILP
- Import module `json` to read example data

In [ ]:
from docplex.mp.model import Model
import json

## Graph-Coloring ILP using Big-M Transformation <font size=3>(variables are colored blue)</font>
**Minimize**
### $$\color{blue}x_\textbf{MAX}$$
**Subject to**
### \begin{align*}
{\color{blue}x_\textbf{MAX}} - {\color{blue}x_{(v)}}&\ge 0,\qquad&\forall v\in \mathcal{V}\\
({\color{blue}x_{(u)}} - {\color{blue}x_{(v)}} + M {\color{blue}t_{(u,v)}}&\ge 1) \wedge ({\color{blue}x_{(v)}} - {\color{blue}x_{(u)}} - M {\color{blue}t_{(u,v)}}\ge 1 - M),\qquad&\forall (u,v)\in \mathcal{E}\\
{\color{blue}x_{(v)}}&\in \{0,...,3\},\qquad&\forall v\in \mathcal{V}\\
{\color{blue}t_{(u,v)}}&\in \{0,1\},\qquad&\forall (u,v)\in \mathcal{E}\\
{\color{blue}x_\textbf{MAX}}&\in\mathbb{R}
\end{align*}

## Import the input from example files
### 36 examples are available: `coloring_ex_1`, `coloring_ex_2`, $\dots$ , `coloring_ex_36`
**<font color='red'>Note: For CPLEX Community Edition, only the first 8 examples</font><font> `coloring_ex_1`, $\dots$ , `coloring_ex_8`</font><font color='red'> are applicable.</font>**

In [ ]:
with open('coloring_data/coloring_ex_7', 'r') as fp:
    colr_data = json.load(fp)
V = list(range(colr_data['n']))
inc_E = colr_data['inc_E']
E = [(int(u),v) for u in inc_E for v in inc_E[u]]
print(f'In this example:\n\t- Number of nodes: {len(V)}\n\t- Number of links: {len(E)}')

### The example above was generated randomly. It might not represent a planar graph as the special feature of the Map-Coloring problem. The number of colors therefore might exceed 4. For this reason, in the following, the domain for the color variable is$${\color{blue}x_{(v)}}\in\{0,\cdots,|\mathcal{V}|-1\},\quad\forall v\in\mathcal{V}$$

## Create the ILP model for Graph-Coloring problem using DOcplex

In [ ]:
colr_ILP = Model(name='Graph Coloring')

## Define decision variables
## \begin{align*}
{\color{blue}x_{(v)}}&\in \{0,...,|\mathcal{V}|-1\},\qquad&\forall v\in \mathcal{V}\\
{\color{blue}t_{(u,v)}}&\in \{0,1\},\qquad&\forall (u,v)\in \mathcal{E}\\
{\color{blue}x_\textbf{MAX}}&\in \mathbb{R}
\end{align*}

In [ ]:
colr_ILP.clear() # This line is optional. Its purpose is to avoid variable duplicates

# Note: since the number of variables might be large, rather than using python
# dictionary comprehension, it saves significant time to tell DOcplex to generate
# the variable dictionary on its own
x = colr_ILP.integer_var_dict(keys=V, ub=len(V)-1, name=lambda v: f'x({v})')
t = colr_ILP.binary_var_dict(keys=E, name=lambda e: f't({e[0]},{e[1]})')

xMAX = colr_ILP.continuous_var(name='xMAX')

## Define big-M

In [ ]:
M = 1e3 # 1 thousand

## Define $\color{blue}x_\textbf{MAX}$ constraints, subscriptable by node indices $${\color{blue}x_\textbf{MAX}} - {\color{blue}x_{(v)}}\ge 0,\qquad\forall v\in \mathcal{V}$$

In [ ]:
# Note: since the number of constraints might be large, it saves significant time to
# first create the constraints outside the problem, then later add them in a batch
cts = (xMAX - x[v] >= 0 for v in V)
ctnames = (f'xMAX({v})' for v in V)

# The ending underscore `_` in the method `add_constraints_`
# specifically tells DOcplex not to return anything
colr_ILP.add_constraints_(cts=cts, names=ctnames)

## Define distinct color constraints, subscriptable by link indices \begin{align*}
({\color{blue}x_{(u)}} - {\color{blue}x_{(v)}} + M {\color{blue}t_{(u,v)}}&\ge 1) \wedge ({\color{blue}x_{(v)}} - {\color{blue}x_{(u)}} - M {\color{blue}t_{(u,v)}}\ge 1 - M),\quad&\forall (u,v)\in \mathcal{E}
\end{align*}

In [ ]:
# Note: since the number of constraints might be large, it saves significant time to
# first create the constraints outside the problem, then later add them in a batch
cts_a = (x[u] - x[v] + M*t[u,v] >= 1 for u,v in E)
cts_b = (x[v] - x[u] - M*t[u,v] >= 1 - M for u,v in E)

ctnames_a = (f'dis_colr_a({u},{v})' for u,v in E)
ctnames_b = (f'dis_colr_b({u},{v})' for u,v in E)

colr_ILP.add_constraints_(cts=cts_a, names=ctnames_a)
colr_ILP.add_constraints_(cts=cts_b, names=ctnames_b)

## Define objective function $$\textbf{minimize}\quad \color{blue}x_\textbf{MAX}$$

In [ ]:
colr_ILP.minimize(xMAX)

## Summarize the model

In [ ]:
colr_ILP.print_information()

## Set termination criteria (stop if either criterion is satisfied)
- Solution time is not longer than 5 minutes.
- Relative optimality gap (i.e., the gap between the tightest LP relaxed bound and the best integral solution found so far) is at most 2%.

In [ ]:
colr_ILP.set_time_limit(5*60) # 5 minutes
_ = colr_ILP.parameters.mip.tolerances.mipgap.set(0.02)

## Solve the ILP

In [ ]:
colr_sol = colr_ILP.solve(log_output=True)

## Display the result

In [ ]:
if colr_sol:
    colr_sol.display()